# Two versions of the data set

In [1]:
#Importing necessary packages

import os
import numpy as np
import sklearn
import pandas as pd
import random

#For plotting
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.manifold import SpectralEmbedding
from sklearn.manifold import TSNE

from sklearn.linear_model import ElasticNet,ElasticNetCV 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_squared_error,r2_score

from sklearn.preprocessing import MinMaxScaler

from scipy import interpolate
from scipy.spatial import ConvexHull
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

path = '/Users/nesli/Desktop/SPRING2022/DSCC465/Project/'
os.chdir(path)

random.seed(265) #seed

In [2]:
data=pd.read_csv('counties_pubhealth_cut_0.6.csv')

In [3]:
data.drop(columns=data.columns[0], axis=1, inplace=True)

In [4]:
data

,fips,state,county,percent_fair_or_poor_health,average_number_of_mentally_unhealthy_days,percent_smokers,percent_adults_with_obesity,food_environment_index,percent_excessive_drinking,num_driving_deaths,...,num_minorities,num_institutionalized_in_group_quarters,percentile_rank_age_65_and_older,percentile_rank_age_17_and_younger,percentile_rank_minorities,percentile_rank_multi_unit_housing,percentile_rank_mobile_homes,percentile_rank_institutionalized_in_group_quarters,percentile_rank_housing_and_transportation,percentile_rank_social_vulnerability
0,01001,Alabama,Autauga,20.882987,4.654031,18.081557,33.300000,7.200000,15.026031,56.0,...,13386,490,0.196400,0.831300,0.633900,0.679100,0.726800,0.125100,0.288100,0.377300
1,01003,Alabama,Baldwin,17.509134,4.304056,17.489033,31.000000,8.000000,17.958310,156.0,...,33560,2911,0.643700,0.447600,0.525300,0.973300,0.538700,0.343800,0.332400,0.275700
2,01005,Alabama,Barbour,29.591802,5.185594,21.999985,41.700000,5.600000,12.844016,30.0,...,14402,2932,0.417400,0.361700,0.904200,0.281400,0.937000,0.942700,0.931200,0.984700
3,01007,Alabama,Bibb,19.439724,4.551699,19.114200,37.600000,7.800000,15.570042,29.0,...,5696,2001,0.258500,0.308800,0.645000,0.407200,0.924900,0.915600,0.666300,0.573700
4,01009,Alabama,Blount,21.745293,4.891455,19.208672,33.800000,8.400000,14.230594,72.0,...,7122,552,0.490900,0.646600,0.423800,0.134400,0.846500,0.151500,0.182700,0.498600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,56041,Wyoming,Uinta,15.537464,3.669805,17.212675,35.800000,7.400000,19.113357,20.0,...,2566,244,0.044600,0.966900,0.421200,0.770500,0.875800,0.215900,0.656200,0.535200
3140,56043,Wyoming,Washakie,15.955971,3.682514,16.859400,28.800000,8.300000,16.523261,5.0,...,1470,176,0.730000,0.800700,0.536100,0.346400,0.511600,0.537400,0.514200,0.592800
3141,56045,Wyoming,Weston,13.934025,3.660440,16.693134,33.300000,7.900000,17.684551,10.0,...,558,356,0.615100,0.307500,0.288800,0.552100,0.861800,0.807700,0.633900,0.271600
3142,KSC,Missouri,Kansas City,16.402077,4.195720,18.714348,32.581804,7.385234,20.842316,656.0,...,324240,16112,0.166922,0.744178,0.675575,0.879288,0.055045,0.329812,0.384675,0.392899


## Remove location data

In [5]:
data_notnormalized=data.copy()

In [6]:
del data_notnormalized['fips']

In [7]:
del data_notnormalized['state']

In [8]:
del data_notnormalized['county']

In [9]:
data_notnormalized

,percent_fair_or_poor_health,average_number_of_mentally_unhealthy_days,percent_smokers,percent_adults_with_obesity,food_environment_index,percent_excessive_drinking,num_driving_deaths,num_uninsured,num_primary_care_physicians,num_dentists,...,num_minorities,num_institutionalized_in_group_quarters,percentile_rank_age_65_and_older,percentile_rank_age_17_and_younger,percentile_rank_minorities,percentile_rank_multi_unit_housing,percentile_rank_mobile_homes,percentile_rank_institutionalized_in_group_quarters,percentile_rank_housing_and_transportation,percentile_rank_social_vulnerability
0,20.882987,4.654031,18.081557,33.300000,7.200000,15.026031,56.0,4093.0,25.0,18.0,...,13386,490,0.196400,0.831300,0.633900,0.679100,0.726800,0.125100,0.288100,0.377300
1,17.509134,4.304056,17.489033,31.000000,8.000000,17.958310,156.0,19119.0,155.0,108.0,...,33560,2911,0.643700,0.447600,0.525300,0.973300,0.538700,0.343800,0.332400,0.275700
2,29.591802,5.185594,21.999985,41.700000,5.600000,12.844016,30.0,2136.0,8.0,9.0,...,14402,2932,0.417400,0.361700,0.904200,0.281400,0.937000,0.942700,0.931200,0.984700
3,19.439724,4.551699,19.114200,37.600000,7.800000,15.570042,29.0,1727.0,11.0,5.0,...,5696,2001,0.258500,0.308800,0.645000,0.407200,0.924900,0.915600,0.666300,0.573700
4,21.745293,4.891455,19.208672,33.800000,8.400000,14.230594,72.0,6333.0,13.0,11.0,...,7122,552,0.490900,0.646600,0.423800,0.134400,0.846500,0.151500,0.182700,0.498600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,15.537464,3.669805,17.212675,35.800000,7.400000,19.113357,20.0,2408.0,9.0,18.0,...,2566,244,0.044600,0.966900,0.421200,0.770500,0.875800,0.215900,0.656200,0.535200
3140,15.955971,3.682514,16.859400,28.800000,8.300000,16.523261,5.0,1052.0,4.0,6.0,...,1470,176,0.730000,0.800700,0.536100,0.346400,0.511600,0.537400,0.514200,0.592800
3141,13.934025,3.660440,16.693134,33.300000,7.900000,17.684551,10.0,710.0,3.0,5.0,...,558,356,0.615100,0.307500,0.288800,0.552100,0.861800,0.807700,0.633900,0.271600
3142,16.402077,4.195720,18.714348,32.581804,7.385234,20.842316,656.0,105281.0,811.0,866.0,...,324240,16112,0.166922,0.744178,0.675575,0.879288,0.055045,0.329812,0.384675,0.392899


In [10]:
data_notnormalized.to_csv('X_same_notnormalized.csv')

## Make location related features numerical

In [11]:
# Checking from the internet the fips codes for the missing counties [1]
# Missouri Kansas City 38000 [2]
data.at[3142,'fips']=38000

In [12]:
# Checking from the internet the fips codes for the missing counties [1]
# NYC 36061 [3]
data.at[3143,'fips']=36061

In [13]:
# Replace state with a unique integer

len(np.unique(data['state']))

51

In [14]:
np.unique(data['state'])

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [15]:
# Replace county name with row index

for i in np.arange(len(data)):
    data.at[i,'county']=str(i)

In [16]:
# Convert the column data type [5]
data["county"] = pd.to_numeric(data["county"])

In [17]:
# From [4], returns the index of the value where the equality is satisfied
[j for j, x in enumerate(np.unique(data['state'])) if x == data.at[3143,'state']][0]

32

In [18]:
np.unique(data['state'])

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [19]:
state_list=list()

for i in np.arange(len(data)):
    state_list.append([j for j, x in enumerate(np.unique(data['state'])) if x == data.at[i,'state']][0])

In [20]:
# Now, make the state column equal to this state_list

data['state']=state_list

In [21]:
data

,fips,state,county,percent_fair_or_poor_health,average_number_of_mentally_unhealthy_days,percent_smokers,percent_adults_with_obesity,food_environment_index,percent_excessive_drinking,num_driving_deaths,...,num_minorities,num_institutionalized_in_group_quarters,percentile_rank_age_65_and_older,percentile_rank_age_17_and_younger,percentile_rank_minorities,percentile_rank_multi_unit_housing,percentile_rank_mobile_homes,percentile_rank_institutionalized_in_group_quarters,percentile_rank_housing_and_transportation,percentile_rank_social_vulnerability
0,01001,0,0,20.882987,4.654031,18.081557,33.300000,7.200000,15.026031,56.0,...,13386,490,0.196400,0.831300,0.633900,0.679100,0.726800,0.125100,0.288100,0.377300
1,01003,0,1,17.509134,4.304056,17.489033,31.000000,8.000000,17.958310,156.0,...,33560,2911,0.643700,0.447600,0.525300,0.973300,0.538700,0.343800,0.332400,0.275700
2,01005,0,2,29.591802,5.185594,21.999985,41.700000,5.600000,12.844016,30.0,...,14402,2932,0.417400,0.361700,0.904200,0.281400,0.937000,0.942700,0.931200,0.984700
3,01007,0,3,19.439724,4.551699,19.114200,37.600000,7.800000,15.570042,29.0,...,5696,2001,0.258500,0.308800,0.645000,0.407200,0.924900,0.915600,0.666300,0.573700
4,01009,0,4,21.745293,4.891455,19.208672,33.800000,8.400000,14.230594,72.0,...,7122,552,0.490900,0.646600,0.423800,0.134400,0.846500,0.151500,0.182700,0.498600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,56041,50,3139,15.537464,3.669805,17.212675,35.800000,7.400000,19.113357,20.0,...,2566,244,0.044600,0.966900,0.421200,0.770500,0.875800,0.215900,0.656200,0.535200
3140,56043,50,3140,15.955971,3.682514,16.859400,28.800000,8.300000,16.523261,5.0,...,1470,176,0.730000,0.800700,0.536100,0.346400,0.511600,0.537400,0.514200,0.592800
3141,56045,50,3141,13.934025,3.660440,16.693134,33.300000,7.900000,17.684551,10.0,...,558,356,0.615100,0.307500,0.288800,0.552100,0.861800,0.807700,0.633900,0.271600
3142,38000,25,3142,16.402077,4.195720,18.714348,32.581804,7.385234,20.842316,656.0,...,324240,16112,0.166922,0.744178,0.675575,0.879288,0.055045,0.329812,0.384675,0.392899


In [22]:
data.dtypes

fips                                                    object
state                                                    int64
county                                                   int64
percent_fair_or_poor_health                            float64
average_number_of_mentally_unhealthy_days              float64
                                                        ...   
percentile_rank_multi_unit_housing                     float64
percentile_rank_mobile_homes                           float64
percentile_rank_institutionalized_in_group_quarters    float64
percentile_rank_housing_and_transportation             float64
percentile_rank_social_vulnerability                   float64
Length: 61, dtype: object

In [23]:
# Check if there is column of type string, but for pandas it is object [6]
[ col  for col, dt in data.dtypes.items() if dt == object]

['fips']

In [24]:
# Convert the column data type [5]
data["fips"] = pd.to_numeric(data["fips"])

In [25]:
data

,fips,state,county,percent_fair_or_poor_health,average_number_of_mentally_unhealthy_days,percent_smokers,percent_adults_with_obesity,food_environment_index,percent_excessive_drinking,num_driving_deaths,...,num_minorities,num_institutionalized_in_group_quarters,percentile_rank_age_65_and_older,percentile_rank_age_17_and_younger,percentile_rank_minorities,percentile_rank_multi_unit_housing,percentile_rank_mobile_homes,percentile_rank_institutionalized_in_group_quarters,percentile_rank_housing_and_transportation,percentile_rank_social_vulnerability
0,1001,0,0,20.882987,4.654031,18.081557,33.300000,7.200000,15.026031,56.0,...,13386,490,0.196400,0.831300,0.633900,0.679100,0.726800,0.125100,0.288100,0.377300
1,1003,0,1,17.509134,4.304056,17.489033,31.000000,8.000000,17.958310,156.0,...,33560,2911,0.643700,0.447600,0.525300,0.973300,0.538700,0.343800,0.332400,0.275700
2,1005,0,2,29.591802,5.185594,21.999985,41.700000,5.600000,12.844016,30.0,...,14402,2932,0.417400,0.361700,0.904200,0.281400,0.937000,0.942700,0.931200,0.984700
3,1007,0,3,19.439724,4.551699,19.114200,37.600000,7.800000,15.570042,29.0,...,5696,2001,0.258500,0.308800,0.645000,0.407200,0.924900,0.915600,0.666300,0.573700
4,1009,0,4,21.745293,4.891455,19.208672,33.800000,8.400000,14.230594,72.0,...,7122,552,0.490900,0.646600,0.423800,0.134400,0.846500,0.151500,0.182700,0.498600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,56041,50,3139,15.537464,3.669805,17.212675,35.800000,7.400000,19.113357,20.0,...,2566,244,0.044600,0.966900,0.421200,0.770500,0.875800,0.215900,0.656200,0.535200
3140,56043,50,3140,15.955971,3.682514,16.859400,28.800000,8.300000,16.523261,5.0,...,1470,176,0.730000,0.800700,0.536100,0.346400,0.511600,0.537400,0.514200,0.592800
3141,56045,50,3141,13.934025,3.660440,16.693134,33.300000,7.900000,17.684551,10.0,...,558,356,0.615100,0.307500,0.288800,0.552100,0.861800,0.807700,0.633900,0.271600
3142,38000,25,3142,16.402077,4.195720,18.714348,32.581804,7.385234,20.842316,656.0,...,324240,16112,0.166922,0.744178,0.675575,0.879288,0.055045,0.329812,0.384675,0.392899


In [26]:
data.to_csv('X_same_notnormalized_withlocation.csv')

## Normalize all the features

In [27]:
# From [7]
# Following from [14]

data_min_max_scaled = data.copy()

for column in data_min_max_scaled.columns:
    data_min_max_scaled[column] = (data_min_max_scaled[column] - data_min_max_scaled[column].min()) / (data_min_max_scaled[column].max() - data_min_max_scaled[column].min())    

In [28]:
data=data_min_max_scaled

In [29]:
data

,fips,state,county,percent_fair_or_poor_health,average_number_of_mentally_unhealthy_days,percent_smokers,percent_adults_with_obesity,food_environment_index,percent_excessive_drinking,num_driving_deaths,...,num_minorities,num_institutionalized_in_group_quarters,percentile_rank_age_65_and_older,percentile_rank_age_17_and_younger,percentile_rank_minorities,percentile_rank_multi_unit_housing,percentile_rank_mobile_homes,percentile_rank_institutionalized_in_group_quarters,percentile_rank_housing_and_transportation,percentile_rank_social_vulnerability
0,0.000000,0.00,0.000000,0.388267,0.561029,0.342101,0.461369,0.720000,0.346707,0.015506,...,0.001816,0.002771,0.196400,0.831300,0.633900,0.679100,0.726800,0.125100,0.288100,0.377300
1,0.000036,0.00,0.000318,0.285624,0.468466,0.325449,0.410596,0.800000,0.487587,0.043699,...,0.004554,0.016464,0.643700,0.447600,0.525300,0.973300,0.538700,0.343800,0.332400,0.275700
2,0.000073,0.00,0.000636,0.653214,0.701619,0.452223,0.646799,0.560000,0.241874,0.008176,...,0.001954,0.016583,0.417400,0.361700,0.904200,0.281400,0.937000,0.942700,0.931200,0.984700
3,0.000109,0.00,0.000955,0.344359,0.533963,0.371122,0.556291,0.780000,0.372844,0.007894,...,0.000773,0.011317,0.258500,0.308800,0.645000,0.407200,0.924900,0.915600,0.666300,0.573700
4,0.000145,0.00,0.001273,0.414500,0.623824,0.373777,0.472406,0.840000,0.308491,0.020017,...,0.000966,0.003122,0.490900,0.646600,0.423800,0.134400,0.846500,0.151500,0.182700,0.498600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,0.999927,1.00,0.998727,0.225641,0.300716,0.317682,0.516556,0.740000,0.543080,0.005357,...,0.000348,0.001380,0.044600,0.966900,0.421200,0.770500,0.875800,0.215900,0.656200,0.535200
3140,0.999964,1.00,0.999045,0.238373,0.304077,0.307754,0.362031,0.830000,0.418641,0.001128,...,0.000199,0.000995,0.730000,0.800700,0.536100,0.346400,0.511600,0.537400,0.514200,0.592800
3141,1.000000,1.00,0.999364,0.176860,0.298239,0.303081,0.461369,0.790000,0.474434,0.002537,...,0.000076,0.002013,0.615100,0.307500,0.288800,0.552100,0.861800,0.807700,0.633900,0.271600
3142,0.672171,0.50,0.999682,0.251945,0.439812,0.359885,0.445514,0.738523,0.626147,0.184663,...,0.043998,0.091127,0.166922,0.744178,0.675575,0.879288,0.055045,0.329812,0.384675,0.392899


In [30]:
del data_min_max_scaled

In [31]:
data.to_csv('X_same_with_location.csv')

In [32]:
del data['fips']

In [33]:
del data['state']

In [34]:
del data['county']

In [35]:
data

,percent_fair_or_poor_health,average_number_of_mentally_unhealthy_days,percent_smokers,percent_adults_with_obesity,food_environment_index,percent_excessive_drinking,num_driving_deaths,num_uninsured,num_primary_care_physicians,num_dentists,...,num_minorities,num_institutionalized_in_group_quarters,percentile_rank_age_65_and_older,percentile_rank_age_17_and_younger,percentile_rank_minorities,percentile_rank_multi_unit_housing,percentile_rank_mobile_homes,percentile_rank_institutionalized_in_group_quarters,percentile_rank_housing_and_transportation,percentile_rank_social_vulnerability
0,0.388267,0.561029,0.342101,0.461369,0.720000,0.346707,0.015506,0.004474,0.003380,0.002061,...,0.001816,0.002771,0.196400,0.831300,0.633900,0.679100,0.726800,0.125100,0.288100,0.377300
1,0.285624,0.468466,0.325449,0.410596,0.800000,0.487587,0.043699,0.021010,0.020954,0.012368,...,0.004554,0.016464,0.643700,0.447600,0.525300,0.973300,0.538700,0.343800,0.332400,0.275700
2,0.653214,0.701619,0.452223,0.646799,0.560000,0.241874,0.008176,0.002321,0.001082,0.001031,...,0.001954,0.016583,0.417400,0.361700,0.904200,0.281400,0.937000,0.942700,0.931200,0.984700
3,0.344359,0.533963,0.371122,0.556291,0.780000,0.372844,0.007894,0.001871,0.001487,0.000573,...,0.000773,0.011317,0.258500,0.308800,0.645000,0.407200,0.924900,0.915600,0.666300,0.573700
4,0.414500,0.623824,0.373777,0.472406,0.840000,0.308491,0.020017,0.006939,0.001757,0.001260,...,0.000966,0.003122,0.490900,0.646600,0.423800,0.134400,0.846500,0.151500,0.182700,0.498600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,0.225641,0.300716,0.317682,0.516556,0.740000,0.543080,0.005357,0.002620,0.001217,0.002061,...,0.000348,0.001380,0.044600,0.966900,0.421200,0.770500,0.875800,0.215900,0.656200,0.535200
3140,0.238373,0.304077,0.307754,0.362031,0.830000,0.418641,0.001128,0.001128,0.000541,0.000687,...,0.000199,0.000995,0.730000,0.800700,0.536100,0.346400,0.511600,0.537400,0.514200,0.592800
3141,0.176860,0.298239,0.303081,0.461369,0.790000,0.474434,0.002537,0.000752,0.000406,0.000573,...,0.000076,0.002013,0.615100,0.307500,0.288800,0.552100,0.861800,0.807700,0.633900,0.271600
3142,0.251945,0.439812,0.359885,0.445514,0.738523,0.626147,0.184663,0.115827,0.109639,0.099175,...,0.043998,0.091127,0.166922,0.744178,0.675575,0.879288,0.055045,0.329812,0.384675,0.392899


In [36]:
data.to_csv('X_same.csv')

## References

[1] https://www.askpython.com/python-modules/pandas/update-the-value-of-a-row-dataframe

[2] https://health.mo.gov/data/geocodes/geocodelist2.php

[3] https://guides.newman.baruch.cuny.edu/nyc_data#:~:text=Manhattan%20is%20New%20York%20County%20(ANSI%20%2F%20FIPS%2036061)

[4] https://stackoverflow.com/questions/6294179/how-to-find-all-occurrences-of-an-element-in-a-list

[5] https://stackoverflow.com/questions/15891038/change-column-type-in-pandas#:~:text=1.-,to_numeric(),floating%2Dpoint%20numbers%20as%20appropriate.

[6] https://stackoverflow.com/questions/43049545/python-check-if-dataframe-column-contain-string-type

[7] https://www.geeksforgeeks.org/data-normalization-with-pandas/